In [1]:
import wandb
import numpy as np
import os
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import argparse
from sklearn import metrics
import torch.nn.functional as F

from datasets.esc50 import get_test_set, get_training_set
from models.mn.model import get_model as get_mobilenet
from models.dymn.model import get_model as get_dymn
from models.preprocess import AugmentMelSTFT
from helpers.init import worker_init_fn
from helpers.utils import NAME_TO_WIDTH, exp_warmup_linear_down, mixup

AssertionError: Specify ESC50 dataset location in variable 'dataset_dir'. Check out the Readme file for further instructions. https://github.com/fschmid56/EfficientAT/blob/main/README.md

In [9]:
import argparse
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

# Import Models
from EfficientAT.models.mn.model import get_model as get_mn
from EfficientAT.models.dymn.model import get_model as get_dymn

# Dataset and Utility Functions
from Datasets import MelSpectrogramDataset
from utils import train_model, evaluate_model

# Argument Parser for Parameters
def parse_args():
    parser = argparse.ArgumentParser(description="ESC-50 Training Configuration")

    # General Parameters
    parser.add_argument("--train_dir", type=str, default="path/to/train", help="Path to training directory")
    parser.add_argument("--test_dir", type=str, default="path/to/test", help="Path to testing directory")
    parser.add_argument("--batch_size", type=int, default=32, help="Batch size for training")
    parser.add_argument("--num_epochs", type=int, default=100, help="Number of training epochs")
    parser.add_argument("--lr", type=float, default=1e-4, help="Learning rate")
    parser.add_argument("--weight_decay", type=float, default=1e-5, help="Weight decay for optimizer")

    # Model Parameters
    parser.add_argument("--model_name", type=str, default="mn10_as", help="Model name (mn10_as or dymn)")
    parser.add_argument("--pretrained", action="store_true", default=False, help="Use pretrained model weights")
    parser.add_argument("--model_width", type=float, default=1.0, help="Model width multiplier")

    # Device Configuration
    parser.add_argument("--cuda", action="store_true", help="Enable CUDA if available")

    return parser.parse_args()


def main():
    # Parse arguments
    import sys
    if "ipykernel" in sys.modules:
        class Args:
            train_dir = "path/to/train"
            test_dir = "path/to/test"
            batch_size = 32
            num_epochs = 100
            lr = 1e-4
            weight_decay = 1e-5
            model_name = "mn10_as"
            pretrained = False
            model_width = 1.0
            cuda = True

        args = Args()
    else:
        args = parse_args()
        

    # Device configuration
    device = torch.device("cuda" if args.cuda and torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Transform: Normalize Mel spectrogram
    transform = transforms.Compose([
        transforms.Lambda(lambda x: (x - np.mean(x)) / (np.std(x) + 1e-6))
    ])

    # Datasets
    train_dataset = MelSpectrogramDataset(
        features_dir=args.train_dir,
        labels_path=os.path.join(args.train_dir, "labels.npy"),
        transform=transform
    )

    test_dataset = MelSpectrogramDataset(
        features_dir=args.test_dir,
        labels_path=os.path.join(args.test_dir, "labels.npy"),
        transform=transform
    )

    # Dataloaders
    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

    # Model Initialization
    if args.model_name.startswith("dymn"):
        model = get_dymn(width_mult=args.model_width, pretrained_name=args.model_name if args.pretrained else None,
                         num_classes=50)
    else:
        model = get_mn(width_mult=args.model_width, pretrained_name=args.model_name if args.pretrained else None)
    
    model.to(device)

    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

    # Train and Evaluate
    train_model(model, train_loader, test_loader, device, optimizer, num_epochs=args.num_epochs)
    test_accuracy = evaluate_model(model, test_loader, device)

    print(f"Test Accuracy: {test_accuracy:.2f}%")

if __name__ == "__main__":
    main()



Using device: cuda


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/train\\labels.npy'

In [3]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

# Import Models
from EfficientAT.models.mn.model import get_model as get_mn
from EfficientAT.models.dymn.model import get_model as get_dymn

# Dataset and Utility Functions
from Datasets import MelSpectrogramDataset
from utils import train_model, evaluate_model

# Training and Testing Directory Paths
train_dir = "C:/Users/jimmy/Desktop/Practical_Work/processed_data/mel_spectrogram/train"
test_dir = "C:/Users/jimmy/Desktop/Practical_Work/processed_data/mel_spectrogram/test"

# Configuration Parameters
batch_size = 32
num_epochs = 100
lr = 1e-4
weight_decay = 1e-5
model_name = "mn10_as"  # Choose "mn10_as" or "dymn"
pretrained = False
model_width = 1.0
cuda_enabled = True

def main():
    # Device configuration
    device = torch.device("cuda" if cuda_enabled and torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Transform: Normalize Mel spectrogram
    transform = transforms.Compose([
        transforms.Lambda(lambda x: (x - np.mean(x)) / (np.std(x) + 1e-6))
    ])

    # Datasets
    train_dataset = MelSpectrogramDataset(
        features_dir=train_dir,
        labels_path=os.path.join(train_dir, "labels.npy"),
        transform=transform
    )

    test_dataset = MelSpectrogramDataset(
        features_dir=test_dir,
        labels_path=os.path.join(test_dir, "labels.npy"),
        transform=transform
    )

    # Dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Model Initialization
    if model_name.startswith("dymn"):
        model = get_dymn(width_mult=model_width, pretrained_name=model_name if pretrained else None, num_classes=50)
    else:
        model = get_mn(width_mult=model_width, pretrained_name=model_name if pretrained else None)
    model.to(device)


    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Train and Evaluate
    train_model(model, train_loader, test_loader, device, num_epochs=num_epochs)
    test_accuracy = evaluate_model(model, test_loader, device)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

if __name__ == "__main__":
    main()
    


Using device: cuda
MN(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), 

RuntimeError: The size of tensor a (527) must match the size of tensor b (32) at non-singleton dimension 1